# Simple MD Example

A simple example of how to run MD simulations is with the molecule, Alanine Dipeptide. We will model the residue, solvate it in a water box, equilibrate the system, and run MD production simulations with AmberTools. This should take roughly an hour.

```{figure} ../../_static/simple.mov
---
width: 50%
---
MD trajectory of Alanine Dipeptide (*ball-and-stick*) in water (*line*) viewed using VMD.
```

The system, Alanine Dipeptide, is the amino acid, Alanine, with the charged ends neutralized. This is done by "capping" the N-terminal with an acetyl group, and C-terminal with N-methylamide.

In this tutorial, we will:

1. Make a new project folder
2. Use `tleap` to make the molecule (in Amber format)
3. Prepare inputs for MD simulations
4. Run MD simulations with `sander`
5. Analyze the trajectory

**This assumes you have already installed some form of conda, and made the conda environment "ambertools".**

````{note}
- This tutorial can be perform on your local computer or with Google Colab
- Download my files for reference: [simple.tar.bz2](../../assets/simple.tar.bz2 "download")
    - To do this on command line:
         1. Right-click the link
         2. Choose copy link
         3. On your terminal, use the command, `wget` followed by pasting the link
         4. Extract the file with `tar xvjf simple.tar.bz2`
````

## Prepare a Project Folder

Make a new project folder, name it `simple`, and then `cd` into it. This will be our working directory.

```bash
mkdir -p simple
cd simple
```

The `-p` option for `mkdir` will report no error if existing, and make parent directories as needed. 

## Prepare Amber Inputs (`.parm7` & `.rst7`)

Alanine dipeptide is an amino acid, so we can use `tleap` to generate the topology (`.parm7`) and coordinate files(`.rst7`). We need to make a script called, "tleap.in", which will be the input to the `tleap` program

```bash
# tleap.in file
source leaprc.protein.ff19SB
source leaprc.water.TIP3P

system = sequence { ACE ALA NME }

solvatebox system TIP3PBOX 12.0 iso 0.8

saveamberparm system step3_pbcsetup.parm7 step3_pbcsetup.rst7

quit
```

Where:

* `source` lines tell Amber which force field parameters we want for our molecules, this is always at the top of the file
    * `leaprc.protein.ff19SB` contains protein information
    * `leaprc.water.TIP3P` has the water parameters
* `system = sequence { ACE ALA NME }` makes use of the `sequence` command which combines amino acid residues together to build a molecule, and this is stored as "system" variable
* `solvatebox system TIP3PBOX 12.0 iso 0.8` create a a water box around the `system`, such that the minimum distance between any atom in `system` and the edge of the periodic box `12.0` Å. 
    * `iso` rotates the `system` to orient the principal axes
    * `0.8` is how close a solvent atom can be to the system atoms
* `saveamberparm system step3_pbcsetup.parm7 step3_pbcsetup.rst7` saves `system` as Amber topology (`step3_pbcsetup.parm7`) and coordinate (`step3_pbcsetup.rst7`) files
* `quit` exits the tleap program.

Now we can run `tleap`:

```bash
tleap -sf tleap.in
```

The `-sf` for `tleap` lets us run the script `tleap.in`, otherwise it will run interactively in the command line.


```bash
tleap -sf tleap.in
```

If you list the files in this directory, you should see `step3_pbcsetup.parm7` and `step3_pbcsetup.rst7`.

```{note}
Visualize these two files with VMD. Always do this before moving on!
```

## MD Inputs - Simulation Settings

Now, we will make the Amber MD input files. These files contain the settings for each MD run. We will have 3 steps:

1. Minimization
2. Heating (for 20 ps from 0 K to 300 K)
3. Production MD (100 ps at 300 K and 1 atm)

::::{tab-set}
::: {tab-item} min.mdin
```bash
Minimize
 &cntrl
  imin=1    
  ntx=1,        
  irest=0,
  maxcyc=2000,
  ncyc=1000,
  ntpr=100,
  ntwx=0,
  cut=8.0,
 /
```
:::
:::{tab-item} heat.mdin
```bash
Heat
 &cntrl
  imin=0,
  ntx=1,
  irest=0,
  nstlim=10000,
  dt=0.002,
  ntf=2,
  ntc=2,
  tempi=0.0,
  temp0=300.0,
  ntpr=100,
  ntwx=100,
  cut=8.0,
  ntb=1,
  ntp=0,
  ntt=3,
  gamma_ln=2.0,
  nmropt=1,
  ig=-1,
 /
&wt type='TEMP0', istep1=0, istep2=9000, value1=0.0, value2=300.0 /
&wt type='TEMP0', istep1=9001, istep2=10000, value1=300.0, value2=300.0 /
&wt type='END' /
```
:::
:::{tab-item} prod.mdin
```bash
Production
 &cntrl
  imin=0,
  ntx=5,
  irest=1,
  nstlim=50000,
  dt=0.002,
  ntf=2,
  ntc=2,
  temp0=300.0,
  ntpr=100,
  ntwx=100,
  cut=8.0,
  ntb=2,
  ntp=1,
  ntt=3,
  barostat=1,
  gamma_ln=2.0,
  ig=-1,
 /
```
:::
::::

## Running the Simulation

Amber has 2 MD engines, `sander` and `pmemd`. `sander` is free and shipped with Ambertools. 

```{note}
You can also run this with the MPI version of `sander` or `pmemd`, or the GPU version of `pmemd` called `pmemd.cuda` if you have access to our Amber software. Just change `sander` to your desired MD engine.
```

Run each step in sequential order, starting with minimization.

::::{tab-set}
:::{tab-item} Step 1 - Minimization
```bash
sander -O -i min.in -p step3_pbcsetup.parm7 -c step3_pbcsetup.rst7 -o min.out -r min.ncrst -inf min.mdinfo
```
:::
::::{tab-item} Step 2 - Heating
:::
```bash
sander -O -i heat.in -p step3_pbcsetup.parm7 -c min.ncrst -o heat.out -r heat.ncrst -inf heat.mdinfo -x heat.nc
```
:::
::::{tab-item} Step 3 - Production MD
:::
```bash
sander -O -i prod.in -p step3_pbcsetup.parm7 -c heat.ncrst -o prod.out -r prod.ncrst -inf prod.mdinfo -x prod.nc
```
:::
::::


## Conclusion

Now you can visual the results with VMD or Chimera, and perform some analysis.